In [151]:
import pandas

#convert the sites into a df
sitesdf = pandas.DataFrame(columns=('icon', 'title', 'url', 'description'))
f = open("hkssites.txt", "r")
row = 0
icon =""
title=""
url=""
desc=""
for x in f:
    x =x.strip()
    if x == "":
        continue
    if len(x) == 1 or len(x) == 2:
        #add previous site and reset
        if  title+url != "":
            sitesdf.loc[row] = [icon,title,url,desc]
            row+=1
            title=""
            url=""
        icon=x    
    if "http" in x:
        url = x
    if url == "":
        title =x
    else :
        desc = x
    
f.close()
sitesdf = sitesdf.reset_index()  # make sure indexes pair with number of rows
print(sitesdf.shape[0])
print(sitesdf.tail(1))

216
     index icon               title   
215    215   CP  Commercial / PLACE  \

                                                   url   
215  https://hksinc.sharepoint.com/sites/Commercial...  \

                                           description  
215  This Group is for Shanghai Commercial/PLACE me...  


In [156]:
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
import os
from bs4 import BeautifulSoup
from dotenv import load_dotenv

load_dotenv(".env")
userCredential = UserCredential(os.getenv('user'),os.getenv('password') )


In [104]:
def sitepages(ctx):
    #list pages relative to site_url
    site_pages = ctx.site_pages.pages.get().execute_query()
    # for site_page in site_pages:  # type: SitePage
    #     print(site_page.file_name)
    print(len(site_pages))
    return site_pages

In [127]:
def content(ctx, relative_url):
    file = ctx.web.get_file_by_server_relative_url(relative_url)
    file_item = file.listItemAllFields.select(["CanvasContent1", "LayoutWebpartsContent"]).get().execute_query()
    
    #print(content)
    return file_item.properties.get("CanvasContent1")

In [158]:


# Create a directory to store the text files
if not os.path.exists("text/"):
        os.mkdir("text/")

if not os.path.exists("text/hksinc.sharepoint.com/"):
        os.mkdir("text/hksinc.sharepoint.com/")

# Create a directory to store the csv files
if not os.path.exists("processed"):
        os.mkdir("processed")
        
#iterate sites        
for index, row in sitesdf.head(1).iterrows():
    site_url = row["url"]
    ctx = ClientContext(site_url).with_credentials(userCredential)
    web = ctx.web
    ctx.load(web)
    ctx.execute_query()
    print(site_url)
    #print(site_url.replace("https://hksinc.sharepoint.com",""))
    for page in sitepages(ctx):
        relative_url = site_url.replace("https://hksinc.sharepoint.com","")+ "/SitePages/"+page.file_name
        
        #get content for site page
        cleanUrl = relative_url.replace("/", "_").replace("?", "_").replace(":", "_").replace(".", "_")
        #print(cleanUrl)
        try:
            t = content(ctx, relative_url)
        except:
            print("no content")
        else:
            if t is not None:
                soup = BeautifulSoup(t)
                    # Get the text but remove the tags
                text = soup.get_text()
                if text !="":
                    with open('text/hksinc.sharepoint.com/'+cleanUrl + ".txt", "w", encoding="utf-8") as f:    
                        

                        # If the crawler gets to a page that requires JavaScript, it will stop the crawl
                        if ("You need to enable JavaScript to run this app." in text):
                            print("Unable to parse page " + url + " due to JavaScript being required")
                        
                        # Otherwise, write the text to the file in the text directory
                        f.write(text)

https://hksinc.sharepoint.com
79
no content


In [143]:
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
site_url = "https://hksinc.sharepoint.com"
ctx = ClientContext(site_url).with_credentials(userCredential)
web = ctx.web
ctx.load(web)
ctx.execute_query()
print("Web title: {0}".format(web.properties['Title'])) 

file = ctx.web.get_file_by_server_relative_url("/SitePages/Breaking-News--Fast-Company-Announces-Project-Connect.aspx")

file_item = file.listItemAllFields.select(["CanvasContent1", "LayoutWebpartsContent","BannerImageUrl"]).get().execute_query()
print(file_item.properties.get("BannerImageUrl"))

Web title: Source
{'Description': 'https://hksinc.sharepoint.com/_layouts/15/getpreview.ashx?guidSite=1d26113b-6e40-4404-814d-038f1605cea0&guidWeb=501cc4e4-f8f8-43a7-9b83-45e8756fa5db&guidFile=1cb893cc-4766-41bc-80a4-22aea687cd9f', 'Url': 'https://hksinc.sharepoint.com/_layouts/15/getpreview.ashx?guidSite=1d26113b-6e40-4404-814d-038f1605cea0&guidWeb=501cc4e4-f8f8-43a7-9b83-45e8756fa5db&guidFile=1cb893cc-4766-41bc-80a4-22aea687cd9f'}


In [115]:
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.webs.web import Web
from office365.sharepoint.files.system_object_type import FileSystemObjectType
site_url = "https://hksinc.sharepoint.com/sites/SpatialAnalyticsModule"
ctx = ClientContext(site_url).with_credentials(userCredential)
web = ctx.web
ctx.load(web)
ctx.execute_query()
print("Web title: {0}".format(web.properties['Title']))
file = ctx.web.get_file_by_server_relative_url("/sites/SpatialAnalyticsModule/SitePages/Overview.aspx")
file_item = file.listItemAllFields.select(["CanvasContent1", "LayoutWebpartsContent"]).get().execute_query()
print(file_item.properties.get("CanvasContent1"))




Web title: Spatial Analytics
<div><div data-sp-canvascontrol="" data-sp-canvasdataversion="1.0" data-sp-controldata="&#123;&quot;controlType&quot;&#58;4,&quot;id&quot;&#58;&quot;9e9bd21c-206b-4b77-981d-06d6e75fb7c7&quot;,&quot;position&quot;&#58;&#123;&quot;layoutIndex&quot;&#58;1,&quot;zoneIndex&quot;&#58;1,&quot;sectionIndex&quot;&#58;1,&quot;controlIndex&quot;&#58;1,&quot;sectionFactor&quot;&#58;12&#125;,&quot;addedFromPersistedData&quot;&#58;true&#125;"><div data-sp-rte=""><p><span class="fontSizeXLarge"><em>This training module provides a structured way to understand the process, rules of engagement,&#160;required time and budget for applying spatial analytics tools in various project phases – from pre-design to occupancy. The tools presented in this module are part of a larger HKS Research Placed-Based Analytics signature approach of layering human, spatial and environmental data to gain insight in the relationship between patterns of people and&#160;space.</em></span></p></div><

In [69]:
doc_lib = ctx.web.lists.get_by_title("Documents")
items = doc_lib.items.select(["FileSystemObjectType"]).expand(["File", "Folder"]).get().execute_query()
for item in items:  # type: ListItem
    if item.file_system_object_type == FileSystemObjectType.Folder:
        print("(Folder): {0}".format(item.folder.serverRelativeUrl))
    else:
        print("(File): {0}".format(item.file.serverRelativeUrl))

(File): /sites/SpatialAnalyticsModule/Shared Documents/02_PhotoEssay_On-Site Documentation_Example.pdf
(File): /sites/SpatialAnalyticsModule/Shared Documents/05_Interviews_Documentation Example.pdf
(File): /sites/SpatialAnalyticsModule/Shared Documents/Visibility-and-Integration.mp4
(File): /sites/SpatialAnalyticsModule/Shared Documents/Proximity-Analysis.mp4
(File): /sites/SpatialAnalyticsModule/Shared Documents/Travel-Distance-Analysis.mp4
(File): /sites/SpatialAnalyticsModule/Shared Documents/_zip_Spatial Analytics Resources and Templates Folder.zip


In [70]:
#list pages relative to site_url
site_pages = ctx.site_pages.pages.get().execute_query()

print(len(site_pages))
for site_page in site_pages:  # type: SitePage
    print(site_page.file_name)
    
    

11
Home.aspx
Overview.aspx
Process.aspx
Synthesis-and-Documentation.aspx
Data.aspx
Fulcrum.aspx
Annotated-Plan.aspx
Photo-Essay.aspx
Shadowing.aspx
Behavior-Mapping.aspx
Interviewing.aspx
